# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyEmbedExternal,
  AppBskyEmbedImages,
  AppBskyFeedDefs,
  AppBskyFeedPost,
  AppBskyRichtextFacet,
} from "@mary/bluesky-client/lexicons";

type UnwrapArray<T> = T extends (infer V)[] ? V : never;

export type Facet = AppBskyRichtextFacet.Main;
export type FacetFeature = UnwrapArray<Facet["features"]>;

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;
type EmbedImages = AppBskyEmbedImages.View;
type EmbedExternal = AppBskyEmbedExternal.View;
type LinkFeature = AppBskyRichtextFacet.Link;
type MentionFeature = AppBskyRichtextFacet.Mention;
// type TagFeature = AppBskyRichtextFacet.Tag;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<[string, string]> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return [`at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`, h.data.did];
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/jason.energy/post/3ldllxneijk2d",
);

[
  "at://did:plc:ga3wlji66r5mxqch6wh7nq4v/app.bsky.feed.post/3ldllxneijk2d",
  "did:plc:ga3wlji66r5mxqch6wh7nq4v"
]

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

type ThreadData = {
  handle: string;
  posts: Post[];
};

function reconstructThread(posts: Post[]): Post[] {
  const sortChronologically = (a: Post, b: Post): number =>
    // @ts-ignore record type is unknown
    new Date(a.record.createdAt).getTime() -
    // @ts-ignore record type is unknown
    new Date(b.record.createdAt).getTime();
  const postMap = new Map<string, Post>();
  const rootPosts: Post[] = [];
  const childrenMap = new Map<string, Post[]>();

  for (const post of posts) {
    postMap.set(post.uri, post);
    // @ts-ignore record type is unknown
    if (!post.record.reply) {
      rootPosts.push(post);
    } else {
      // @ts-ignore record type is unknown
      const parentUri = post.record.reply.parent.uri;
      if (!childrenMap.has(parentUri)) {
        childrenMap.set(parentUri, []);
      }
      childrenMap.get(parentUri)!.push(post);
    }
  }

  rootPosts.sort(sortChronologically);

  for (const children of childrenMap.values()) {
    children.sort(
      sortChronologically,
    );
  }

  const result: Post[] = [];

  function addPostAndReplies(post: Post) {
    result.push(post);
    const children = childrenMap.get(post.uri) || [];
    for (const child of children) {
      addPostAndReplies(child);
    }
  }

  for (const rootPost of rootPosts) {
    addPostAndReplies(rootPost);
  }

  return result;
}

export const downloadThread = async (
  postUrl: string,
): Promise<ThreadData> => {
  const [uri, handle] = await postURLToAtpURI(postUrl);
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri,
    },
  });
  return {
    handle,
    posts: reconstructThread(unwrapThreadPosts(d.data.thread as Thread)),
  };
};

In [ ]:
const { posts, handle } = await downloadThread(
  "https://bsky.app/profile/callmephilip.com/post/3lr62ephhac27",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/unicode.ts

const encoder = new TextEncoder();
const decoder = new TextDecoder();

interface UtfString {
  u16: string;
  u8: Uint8Array;
}

const createUtfString = (utf16: string): UtfString => {
  return {
    u16: utf16,
    u8: encoder.encode(utf16),
  };
};

const getUtf8Length = (utf: UtfString) => {
  return utf.u8.byteLength;
};

const sliceUtf8 = (utf: UtfString, start?: number, end?: number) => {
  return decoder.decode(utf.u8.slice(start, end));
};

// replace

In [ ]:
//| export

const externalEmbedToMarkdown = (
  embed: EmbedExternal,
  authorDid: string,
): string => {
  const { title, description, uri, thumb } = embed.external;

  console.log(embed);

  let thumbUrl: string | undefined;

  if (typeof thumb === "string") {
    thumbUrl = thumb;
  } else if (thumb) {
    thumbUrl =
      // @ts-ignore not sure why this is failing
      `https://cdn.bsky.app/img/feed_thumbnail/plain/${authorDid}/${thumb.ref.$link}@jpeg`;
  }

  const thumbImage = thumbUrl ? `![${title}](${thumbUrl})` : null;

  return [`### [${title}](${uri})`, description, thumbImage || ""]
    .map((l: string) => `> ${l}`)
    .join("\n");
};

In [ ]:
//| export

// from https://github.com/mary-ext/skeetdeck/blob/aa0cb74c0ace489b79d2671c4b9e740ec21623c7/app/api/richtext/segmentize.ts

export interface RichtextSegment {
  text: string;
  feature: FacetFeature | undefined;
}

const createSegment = (
  text: string,
  feature: FacetFeature | undefined,
): RichtextSegment => {
  return { text: text, feature: feature };
};

export const segmentRichText = (
  rtText: string,
  facets: Facet[] | undefined,
): RichtextSegment[] => {
  if (facets === undefined || facets.length === 0) {
    return [createSegment(rtText, undefined)];
  }

  const text = createUtfString(rtText);

  const segments: RichtextSegment[] = [];
  const length = getUtf8Length(text);

  const facetsLength = facets.length;

  let textCursor = 0;
  let facetCursor = 0;

  do {
    const facet = facets[facetCursor];
    const { byteStart, byteEnd } = facet.index;

    if (textCursor < byteStart) {
      segments.push(
        createSegment(sliceUtf8(text, textCursor, byteStart), undefined),
      );
    } else if (textCursor > byteStart) {
      facetCursor++;
      continue;
    }

    if (byteStart < byteEnd) {
      const subtext = sliceUtf8(text, byteStart, byteEnd);
      const features = facet.features;

      if (features.length === 0 || subtext.trim().length === 0) {
        segments.push(createSegment(subtext, undefined));
      } else {
        segments.push(createSegment(subtext, features[0]));
      }
    }

    textCursor = byteEnd;
    facetCursor++;
  } while (facetCursor < facetsLength);

  if (textCursor < length) {
    segments.push(
      createSegment(sliceUtf8(text, textCursor, length), undefined),
    );
  }

  return segments;
};

const processFacets = (text: string, facets: Facet[] | undefined): string => {
  return segmentRichText(text, facets).reduce((acc, segment) => {
    const { text, feature } = segment;

    if (feature?.$type === "app.bsky.richtext.facet#link") {
      return acc + `[${text}](${feature.uri})`;
    } else if (feature?.$type === "app.bsky.richtext.facet#mention") {
      return acc + `[${text}](https://bsky.app/profile/${feature.did})`;
    }

    return acc + segment.text;
  }, "");
};

export const postToMd = (post: Post, handle: string): string => {
  const record = post.record as PostRecord;
  const text = record.text;
  let richtext = text;
  let embeds = "";

  richtext = processFacets(text, record.facets);

  if (post.embed) {
    if (post.embed.$type === "app.bsky.embed.images#view") {
      const e = post.embed as EmbedImages;
      for (const image of e.images) {
        embeds += `![${
          image.alt || "no image description"
        }](${image.fullsize})\n`;
      }
    } else if (post.embed.$type === "app.bsky.embed.external#view") {
      embeds += externalEmbedToMarkdown(
        post.embed as EmbedExternal,
        handle,
      );
    }
  }

  const [d, t] = record.createdAt.split("T");
  const [h, m] = t.split(":");

  const r = [
    `> [${post.author.displayName} - @${post.author.handle}](https://bsky.app/profile/${post.author.handle}) **${d} ${
      [h, m].join(
        ":",
      )
    }**`,
    ...richtext.split("\n").filter((l: string) => l.trim() !== ""),
    embeds,
  ].join("\n\n");

  return r;
};

// await Deno.jupyter.display(
//   {
//     "text/markdown": postToMd(posts[4], handle),
//   },
//   { raw: true }
// );

Support links inside posts

In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const { posts, handle } = await downloadThread(postUrl);
  return posts.map((post) => postToMd(post, handle)).join("\n\n");
};

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": await downloadPostToMd(
      "https://bsky.app/profile/callmephilip.com/post/3lr62ephhac27",
    ),
  },
  { raw: true },
);

> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:19**

🧵 Notes on `claude.md` structure and best practices



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:19**

import additional files using @path/to/import syntax

```

See @README for project overview and @package.json for available npm commands for this project.

# Additional Instructions

- git workflow @docs/git-instructions.md

```



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:19**

importing files in user’s home dir is a convenient way for your team members to provide individual instructions that are not checked into the repository:

```

# Individual Preferences

- @~/.claude/my-project-instructions.md

```



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:19**

imports are not evaluated inside markdown code spans and code blocks:

```

This code span will not be treated as an import: `@anthropic-ai/claude-code`

```



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:19**

Imported files can recursively import additional files, with a max-depth of 5 hops. You can see what memory files are loaded by running /memory command.



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-09 09:57**

`claude.md` from claude code github action repo - [github.com/anthropics/c...](https://github.com/anthropics/claude-code-action/blob/main/CLAUDE.md)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 10:58**

Claude code best practices - [www.anthropic.com/engineering/...](https://www.anthropic.com/engineering/claude-code-best-practices)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:02**

You can also use `CLAUDE.local.md` (and add it to .gitignore) to have your own flavour of `CLAUDE.md`



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:03**

for monorepos, you might run claude from root/foo, and have `CLAUDE.md` files in both root/CLAUDE.md and root/foo/CLAUDE.md. Both of these will be pulled into context automatically



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:04**

`~/.claude/CLAUDE.md` applies to ALL of your claude sessions



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:06**

You should occasionally run your `CLAUDE.md`s through the prompt improver ([docs.anthropic.com/en/docs/buil...](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/prompt-improver)) to improve adherence



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:10**

Review and curate tools that are available to Claude using `--allowedTools` flag `allowed_tools` param in GH action



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:15**

You can check in `.mcp.json` file to list MCP servers available to Claude code. "When working with MCP, it can also be helpful to launch Claude with the --mcp-debug flag to help identify configuration issues." Here's an example in the wild - [github.com/unchainedsho...](https://github.com/unchainedshop/unchained/blob/master/.mcp.json)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:22**

"Paste specific URLs alongside your prompts for Claude to fetch and read. To avoid permission prompts for the same domains (e.g., `docs.foo.com`), use /permissions to add domains to your allowlist."



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:25**

Claude code for issue triage using GH action - very neat. [github.com/anthropics/c...](https://github.com/anthropics/claude-code/blob/main/.github/actions/claude-issue-triage-action/action.yml). Creates a temp prompt file on the fly and then feed it to Dr Shannon



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:34**

example `claude.md` from [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude)

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreigc7sf4mxxz5z3jqekoohtrclwropmbfakjjuxyiwv7ohee44ass4@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:38**

not a huge fan

```

### Code Style  

- Formatting: Prettier with 100-char lines  

- Imports: sorted with simple-import-sort  

- Components: Pascal case, co-located with their tests  ...

```

most of this stuff can adjusted using deterministic formatting tools chained with claude code invocations



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-07-01 08:16**

Claude code has hooks for this stuff re: [bsky.app/profile/call...](https://bsky.app/profile/callmephilip.com/post/3lsv2h4inps2j)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:40**

Here's a very detailed `Claude.md` template from julep [github.com/julep-ai/jul...](https://github.com/julep-ai/julep/blob/dev/AGENTS.md)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 11:48**

Using easily greppable anchor comments in the codebase and refer the them in the guidelines ([diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude))

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreiapuf4kopxgrio45lkwkh4fgrek4t53h4t6u7z2uwmqvwys3qsaga@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:22**

For large code bases set boundaries both in `CLAUDE.md` and locally in your code (via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude)). Side note: how sacred is sacred? Is Claude catholic?

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreie6dxovakbh3kafi252ssqwmxzelxhfqcoqdntajt7ubrxyfetzne@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:29**

A good illustration of "things not to do section" for `CLAUDE.md` (via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude))

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreidhk3ta4o3o6xp36a34zayi6wy5no4b7xyqjombkzufxibqfasaua@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:31**

"As your codebase grows, `CLAUDE.md` alone isn’t enough. You need [...] anchor comments. These serve as local context that prevents AI from making locally bad decisions" - via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude)

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreiefzzl4mjck4axd4zm5iqzw4q2pkbm3jjebyaset753xfmzp4uhke@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:38**

Folks from julep do NOT let Claude write tests: "If an AI tool touches a test file, the PR gets rejected. No exceptions." (via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude))

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreiguxlsu7sz26k5f2k657gxyfjt4hyk5o23o45otrvbdlh2fy65z4a@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:44**

"[...] being stingy with context to save tokens actually costs you more. [...] Front-load context to avoid iteration cycles. Think of tokens like investing in good tools—the upfront cost pays for itself many times over." - via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude)

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreibrv6lwjdzdyxnnmrelzxkhzjub5jydrjd2qw3k4dqu623b34bqgu@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 12:55**

prod case study: combine well-maintained `CLAUDE.md` + `SPEC.md` + prompt [bsky.app/profile/call...](https://bsky.app/profile/callmephilip.com/post/3lrawstzgbk2o)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-10 13:01**

Things Claude CANNOT touch (via [diwank.space/field-notes-...](https://diwank.space/field-notes-from-shipping-real-code-with-claude)) - cool usage of inline comments combined with global directives in `CLAUDE.md`

- test files

- DB migrations

- Security critical code

- API contracts without versioning

- Configuration and secrets

![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreiavwczeactd65toh6ga6g6pzt76bamah6waezfctp2sul5jpc3bbu@jpeg)
![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreidrmbelenqbity74hj572hykqcbtyslfrpyb2pze74nxz2ywf47ti@jpeg)
![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreiav72edxnai5qhqq7arp5ju7ezr4ixruvxppwlulbe5fkoncgm56a@jpeg)
![no image description](https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ubdeopbbkbgedccgbum7dhsh/bafkreid547ux5xp5ztytsezkdjftzs4qvdy7fhl67ysy636uhjjhqy6ab4@jpeg)


> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-23 13:52**

Example `CLAUDE.md` from "A Python library to parse malformed XML" - [github.com/mitsuhiko/sl...](https://github.com/mitsuhiko/sloppy-xml-py/blob/main/CLAUDE.md)  + a writeup [lucumr.pocoo.org/2025/6/21/my...](https://lucumr.pocoo.org/2025/6/21/my-first-ai-library/) via [@mitsuhiko.at](https://bsky.app/profile/did:plc:yym5dkfbnzf6lspvh4hnstjg)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-23 14:32**

re: [bsky.app/profile/simo...](https://bsky.app/profile/simonwillison.net/post/3ls5tcivgfc2o)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-06-29 20:37**

from [bsky.app/profile/mits...](https://bsky.app/profile/mitsuhiko.at/post/3lspd5bj6kc2e) - instruct Claude how to write "throw away" bespoke scripts, where to put them and how to run them - these are tools that Claude can then run to accomplish tasks (reduce reliance on static MCPs)



> [Philip Nuzhnyi - @callmephilip.com](https://bsky.app/profile/callmephilip.com) **2025-07-01 06:17**

You can use hooks inside ‘.claude/settings.json’ to run shell commands deterministically at different stages of the lifecycle. This will likely remove the need for extra mansplaining in ‘claude.md’ [docs.anthropic.com/en/docs/clau...](https://docs.anthropic.com/en/docs/claude-code/hooks) h/t [@fry69.dev](https://bsky.app/profile/did:plc:3zxgigfubnv4f47ftmqdsbal)

